# 자료 적재

In [1]:
import os
import numpy as np
import pandas as pd
rating_file_path=os.getenv('HOME') + '/mini_projects/_E-08_recommend/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# 결측치, 중복 데이터 확인

In [2]:
len(ratings) - ratings.count()

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

In [3]:
ratings[ratings.duplicated()]

,user_id,movie_id,rating,timestamp


# 데이터 전처리

In [4]:
# 3점 이상만 남깁니다.
#ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')  # 왜 timestamp를 없애면 안줄어드는 거지?
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 1000209
Ratio of Remaining Data is 100.00%


In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.                                    # 왜 바꾸는 거지? 평점이 아니라 본 횟수로 간주하려고?
ratings.rename(columns={'rating':'count'}, inplace=True)

In [6]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000204    1
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 1000209, dtype: int64

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/mini_projects/_E-08_recommend/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게하기 위해 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
5,6,heat (1995),Action|Crime|Thriller
6,7,sabrina (1995),Comedy|Romance
7,8,tom and huck (1995),Adventure|Children's
8,9,sudden death (1995),Action
9,10,goldeneye (1995),Action|Adventure|Thriller


In [9]:
movies['genre'] = movies['genre'].str.lower() # 검색을 쉽게하기 위해 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy
5,6,heat (1995),action|crime|thriller
6,7,sabrina (1995),comedy|romance
7,8,tom and huck (1995),adventure|children's
8,9,sudden death (1995),action
9,10,goldeneye (1995),action|adventure|thriller


In [10]:
condition = (ratings['user_id']== ratings.loc[0, 'user_id'])
ratings.loc[condition]

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


# 데이터 탐색

In [11]:
# 유저 수
ratings['user_id'].nunique()

6040

In [12]:
# 유저가 본 영화 수
ratings['movie_id'].nunique()

3706

In [13]:
# 인기 많은 영화
movie_count = ratings.groupby('movie_id')['user_id'].count()        # 영화 이름이 나오게 하려면 어떻게 해야 되지?
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3428
260     2991
1196    2990
1210    2883
480     2672
2028    2653
589     2649
2571    2590
1270    2583
593     2578
1580    2538
1198    2514
608     2513
2762    2459
110     2443
2396    2369
1197    2318
527     2304
1617    2288
1265    2278
1097    2269
2628    2250
2997    2241
318     2227
858     2223
356     2194
2716    2181
296     2171
1240    2098
1       2077
Name: user_id, dtype: int64

In [14]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: movie_id, dtype: float64

In [15]:
# 유저별 평점 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['count'].median()
user_median.describe()

count    6040.000000
mean        3.840811
std         0.577449
min         1.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

In [16]:
mov = [1986, 1013, 1012, 1014, 2021]  # 영화 이름이 나오게 하려면 어떻게 해야 되지?

for i in mov:
    print(movies['title'][i])

hot lead and cold feet (1978)
so dear to my heart (1949)
sword in the stone, the (1963)
robin hood: prince of thieves (1991)
rescuers, the (1977)


In [17]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['3245' , '1234' ,'2367' ,'34' ,'657']

# 'joshua'이라는 user_id가 위 영화를 4번 봤다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['lee']*5, 'movie_id': my_favorite, 'count':[4]*5,})
    
if not ratings.isin({'user_id':['lee']})['user_id'].any(): 
    ratings = ratings.append(my_playlist)


ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,movie_id,count,timestamp
1000204,6040,1091,1,956716541.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,lee,3245,4,NaN
1,lee,1234,4,NaN
2,lee,2367,4,NaN
3,lee,34,4,NaN
4,lee,657,4,NaN


In [18]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [19]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['lee'])  


6040


In [20]:

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('artist column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
artist column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6040,3706,4,NaN
1,6040,3707,4,NaN
2,6040,3708,4,NaN
3,6040,3709,4,NaN


# matrix factorize

In [21]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()



In [22]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6041x3711 sparse matrix of type '<class 'numpy.int64'>'
	with 1000214 stored elements in Compressed Sparse Row format>

# AlternatingLeastSquares

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3711x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 1000214 stored elements in Compressed Sparse Column format>

In [26]:
csr_data_transpose.shape

(3711, 6041)

# 모델 훈련

In [27]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 

In [28]:
lee, toy_story = user_to_idx['lee'], movie_to_idx[1]
lee_vector, toystory_vector = als_model.user_factors[lee], als_model.item_factors[toy_story]

print('슝=3')    

슝=3


In [29]:
print(lee_vector)
print(len(lee_vector))
lee_vector.shape

[-1.3621460e-03 -2.4052055e-05 -2.0414586e-04  3.2099250e-03
 -5.9242820e-04 -3.5410516e-03  3.2743163e-04  3.0114972e-03
 -6.5114687e-04  7.8262226e-04 -2.0587188e-03  1.5148971e-03
  1.9029483e-03 -5.7427026e-03  3.5145332e-03 -3.4488565e-03
 -1.9839325e-03  3.0636964e-03 -8.1950886e-04 -5.1794648e-03
  3.5300725e-03  7.0552388e-03  1.7602877e-03  5.6679669e-04
  2.9978368e-03  1.1029172e-03 -9.9486124e-04 -8.4469153e-04
  2.5253382e-03 -5.3960681e-03  3.4732424e-04 -1.9855741e-03
  1.0668811e-03  4.0057553e-03 -4.7407034e-03  4.0828935e-03
 -2.8317156e-03 -2.2182020e-03  2.0434202e-03 -3.2197714e-03
  1.1202482e-03  1.5250711e-03 -2.0086751e-03  5.6130858e-04
 -8.2110311e-04  2.5638132e-03 -7.8286650e-03  2.7543774e-03
 -4.1146777e-03 -2.1245314e-03 -4.1567258e-04  5.7083177e-03
  1.6662504e-03 -6.9953874e-04 -1.1528959e-03  1.3971117e-03
  1.8086468e-03  3.0611121e-04  1.6234594e-04  6.8741580e-03
  3.2103960e-03  1.1098226e-02  1.7043244e-03 -2.2200854e-03
  4.3402976e-04 -5.72549

(100,)

In [30]:
print(toystory_vector)
len(toystory_vector)
toystory_vector.shape

[ 0.02722646  0.01185132 -0.00138563  0.00715574  0.02311376  0.01301323
 -0.01948186 -0.00524573  0.00585014  0.00538858  0.01569814 -0.01123333
  0.01947936  0.00646068  0.00710667  0.02174431 -0.00250337  0.01846508
  0.02568952  0.02840773  0.02700961  0.03813925 -0.0004978   0.01442924
  0.01201556  0.00748176 -0.01162262 -0.00333789  0.02874931  0.00923476
  0.04118547  0.01496085 -0.02515573 -0.01577307  0.02283118  0.00562489
 -0.01439902 -0.02371249  0.03880126 -0.00564124 -0.02094697 -0.05232745
  0.02545934  0.00532863 -0.00861748 -0.00086329 -0.01597595 -0.00544533
 -0.00114072  0.01895739 -0.01734653  0.00138401  0.01623979  0.00504562
 -0.01502333 -0.01384543 -0.00746733  0.03076619 -0.03940547 -0.01568567
 -0.02937944  0.02976091 -0.00981539  0.00996317  0.00460795  0.0003542
 -0.00647141 -0.01084322  0.00793002 -0.01210472 -0.00957751  0.00256144
  0.01555518 -0.00142447  0.0237728   0.02965154 -0.01940695 -0.0062965
  0.01854683  0.01469565  0.00429153  0.00994107  0.0

(100,)

In [31]:
np.dot(lee_vector, toystory_vector)

0.0003476785

# 영화추천

In [32]:
user = user_to_idx['lee']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended
# 평점 3점 미만 버렸을 때
# factor 가 1000일 땐 0.04 ~ 0.05
# factor 가 500 일 땐 0.02
# factor 가 100일 땐 0.001 ~ 0.002

# 평점제한 없앴을 때
# 1000, 0.03~0.05
# 500, 0.02
# 100, 0.002

# 내 가설이 맞았다. 영화추천에선 평점 제한이 있으나 없으나 별 차이 없다.  그나저나 왜 이렇게 낮은거지?

[(38, 0.0011401094),
 (770, 0.0010288865),
 (2773, 0.0009960156),
 (295, 0.0009936201),
 (165, 0.0009634982),
 (2439, 0.0009618643),
 (2444, 0.00095867447),
 (716, 0.0009485519),
 (257, 0.00091931643),
 (1653, 0.0009184536),
 (752, 0.0009058658),
 (2617, 0.0009043785),
 (174, 0.0008896879),
 (1150, 0.0008656691),
 (2508, 0.0008566374),
 (245, 0.00084992725),
 (2610, 0.0008494856),
 (732, 0.00084617164),
 (2529, 0.00084557733),
 (3361, 0.0008364015)]

In [33]:
for i in movie_recommended:
    print(movies['title'][i[0]])

clueless (1995)
independence day (id4) (1996)
best laid plans (1999)
pushing hands (1992)
feast of july (1995)
breaks, the (1999)
pet sematary (1989)
great white hype, the (1996)
star wars: episode iv - a new hope (1977)
deconstructing harry (1997)
striptease (1996)
red violin, the (le violon rouge) (1998)
living in oblivion (1995)
farmer & chase (1995)
metroland (1997)
houseguest (1994)
finding north (1999)
thinner (1996)
pushing tin (1999)
death wish (1974)


In [36]:

explain = als_model.explain(user, csr_data, itemid=38)
explain

(0.0006987633859626677,
 [(3709, 0.00018972655760196377),
  (3710, 0.00015160942120187837),
  (3706, 0.00015016342549517976),
  (3707, 0.00012713386463054726),
  (3708, 8.013011703309855e-05)],
 (array([[0.6021488 , 0.07919306, 0.12176069, ..., 0.0782789 , 0.10003129,
          0.10199424],
         [0.04768601, 0.61862233, 0.1008575 , ..., 0.10138097, 0.07792505,
          0.11707291],
         [0.07331805, 0.0720353 , 0.63605884, ..., 0.09478665, 0.05118011,
          0.04074657],
         ...,
         [0.04713555, 0.06891568, 0.08004621, ..., 0.52474298, 0.0146327 ,
          0.02576906],
         [0.06023372, 0.05612796, 0.05259277, ..., 0.05432659, 0.51008123,
          0.00867656],
         [0.06141571, 0.08050115, 0.05014378, ..., 0.07504615, 0.06655154,
          0.52634135]]),
  False))

# 비슷한 영화 찾기

In [34]:

similar_movie = als_model.similar_items(movie_to_idx[5], N=15)
print(similar_movie)
print(type(similar_movie))

[(1334, 1.0000001), (1091, 0.74425906), (1772, 0.74174464), (401, 0.7276008), (607, 0.7112133), (1415, 0.69398856), (1321, 0.68053913), (621, 0.67733455), (1866, 0.676931), (634, 0.6638844), (1221, 0.65861034), (627, 0.6580581), (699, 0.65436625), (566, 0.6508656), (744, 0.6504126)]
<class 'list'>


#  프로젝트 진행 중 의문점   
   
### **1. 데이터 전처리 과정에서**   
   
1. 왜 timestamp를 없애면 3점 이상의 평점 필터링이 작동하지 않는 걸까?   
timestamp 칼럼의 정보가 하는 역할이 뭐지?   
   
   
2. rating을 count로 바꾸는 건 평점 정보를 영화를 본 횟수를 의미하게 하기 위해서다.   
그런데 이렇게 바꾸는 바람에   
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))   
여기서 rating.count라고 쓰면 오류가 난다.   
내 생각엔 변수이름 지정이 좋지 않아 보인다.   

### **2. 데이터 탐색에서**   
   
1. 평점이 높은 영화를 뽑아내는 코드에서  영화 제목을 뽑아내는 데 실패했다.   
rating파일엔 평점 정보와 영화 번호가 저장돼 있고   
movie파일엔 영화번호와 영화제목 정보가 저장돼 있다.   
영화 제목을 문자열로 입력하면 영화 평점이 뜨게 하는 함수를 만들어야 한다. **만들어 낼 것**   
   
   
   
2. uset_to_indx, movie_to_idx 코드는 불필요한 것 같다. 이미 인덱싱이 돼 있기 때문이다.   
   
### **3. matrix factorize 단계에서**   
csr_matrix의 shape가 전처리한 데이터와 차이가 없었다.   
별도의 함수 명령을 줘야 원래의 sparse행렬을 압축한 csr mtrix를 확인할 수 있는 것 같다. **확인해 볼 것**   
   
   
### **4. 훈련 후 평가 단계에서**   
   
1. 유저 벡터와 영화 벡터의 내적 값이 너무 작게 나오는 것 같다. 0.002에서 0.05 사이면 1이랑은 한참 멀다.   
일부러 평점이 4점인 영화들로만 골라서 했는데도 이렇게 나오는 건 왜일까?   
   
   
   
2. 나는 평점 3점 이상만 고려하는 게 잘못됐다고 생각했다.   
평점이 아니라 본 횟수를 고려할 경우,      
한 번만 본 것도 추천을 위한 정보로 쓰일만한 가치가 있다   
뮤지션 추천이나 음악추천의 경우 단 한 번만 들은 앨범이나 곡이라 해도   
그것이 유사도가 높은 유저 사이에서의 어느 한쪽의 청음 기록이라고 한다면      
그것은 다른 한쪽에게 추천의 근거가 될 수 있다.      
   
더군다나 여기 영화추천에선   
모든 데이터들이 최대 5번까지 본 것으로 간주되므로      
이러한 분포에서 1번의 시청기록은 큰 의미를 지닐 수 있다.   
따라서 3점이상의 기록만 학습에 쓰도록 하게 되면   
이로부터 얻어낸 벡터 정보는 큰 힘을 갖지 못할 것 같다고 생각했다.      
그래서 나는 평점이 3점 이상만 있는 데이터일 경우와       
평점 제한을 없엔 데이터일 경우, 이렇게 두 가지 경우로 나누어 추천을 받았을 때   
추천 받은 영화에 기여한 다른 평점의 기여도가 다를 거라고 추측했다.   
   
    
그래서 영화 데이터 중, 사람들이 가장 많이 본 영화 TOP10 에서 5개를 뽑아   
나의 시청기록 데이터로 만든 후 모델의 factor수를 달리해 실험해보았다.      
그런데 결과는 두 가지 경우에서 모두 똑같이 나왔다.   
아마 그 이유는 평점의 분포라든가, 평점의 스케일이 1에서 5까지 밖에 안된다는 점,   
각 유저들이 본 영화들의 평점간의 관계를 깊게 의미화하기엔   
주어진 정보가 너무 없다는 점 등 때문이라고 생각한다.   
   
또 다른 한 편으로는, 단 한번의 기록이 추천의 근거가 될 수 있다고 생각하는 내 주장의 근거가   
모델에 구현되어 있지 않았기 때문일 수도 있다.   





# 프로젝트 진행 중 어려웠던 점

아직 데이터 전처리 과정에서 판다스를 다루는 게 숙련되지 않아   
굉장히 기본적인 처리도 못하고 있다.   
그리고 몸도 별로 좋지 않다. 어서 자고 싶다.   

# 회고   
   
딥러닝에는 CNN과 RNN만 있는 줄 알았다.   
추천도 딥러닝으로 한다는 게 신기했다.   
   
   
   
추천 시스템이라는 건 일상에서 많이 사용해봤지만   
사실 나한테 크게 와닿은 적은 없었다.   
내 취향이 까다로워서일 수도 있고   
인공지능의 추천시스템이 아직 충분히 정교하지 않아서일 수도 있고   
내가 제대로 추천받을 수 있을 만큼 충분한 정보를 서비스에 제공하지 않아서일 수도 있고   
뭐랄까, 감히 컴퓨터 나부랭이가 내 취향을 파악해서 내가 좋아할만한 걸 추천해준다고? 하는 폄하가 있어서 일 수도 있다.   
   
굉장히 간단하고 그래서 강력하지 않은 추천시스템을 아주 수박 겉핥기로 살펴본 것에 지나지 않지만,   
추천 시스템이라는 게 굉장히 머리가 좋은 사람이 해야하는 거라는 건 바로 직감했다.   
그 추천 전략을 짜는 것의 대부분은 포장기술에 지나지 않을 것이고   
정말 혁신적인 추천 알고리즘은 굉장히 드물다는 업계 특성도 살짝 느껴졌다.    
제한된 정보한 가지고 뭔가를 추천하려면 약과 MSG를 무지 섞어 팔아야하겠지.   
   
사실 난 추천 시스템에 굉장히 비판적이고 회의적이다.   
내 유튜브 앱에는 시청기록과 검색기록이 꺼져 있은지가 수년이 되었고,   
멜론의 추천시스템은 너무 조악하기 그지 없기도 하거니와 나를 편협한 대중가요에 묶어두게 되는 결과가 너무 빤히 보여   
나만의 플레이 리스트를 쓴지 오래다. 그마저도 그걸 기반으로 추천을 해주는 시스템도 있을 테니   
각각의 플레이리스트에 일부러 패턴이 다른 음악을 넣기도 한다.   
사용자가 자신의 후기를 바로 옆에서 말하는 수준의, 추천 받는 동안 스포를 당하거나 너무 큰 기대를 하지 않게 되거나  
너무 간과하지 않을 정도로 굉장히 섬세하게 표현하는 수준의 추천시스템을 만드려면 어떻게 해야 할까?   
단순히 데이터만을 가지고 수치로 말하는 추천은  
사실 난 잘 모르겠다.   
이건 이미지 분류와 자연어 처리 문제랑은 별개의 문제다.   
나에게 무언갈 추천할 순 있다.   
하지만 그 서비스를 나에게 제공하려면,   
서비스 제공자는 어떤 알고리즘으로 어떤 데이터에 기반해 무슨 근거로 그걸 나에게 추천했는지   
추천하는 즉시 나에게 그 까닭을 제공해야할 의무가 있다고 본다.   
그게 음악이든, 영화든, 특정 정치인이든, 뭐든 간에 말이다.   
개발자가 implicit, explicit한 평가 정보를 바탕으로   
자기 임의로 만들어낸 추천전략에 툭 튀어나와 내앞에 던져진 결과를   
일방적으로 수용하기엔   
내가 도저히 가두어질 마음이 없다.
   
딥러닝의 다른 분야에 비해 기교와 지능이 훨씬 더 많이 필요한 영역임은   
이번 단 한번의 기회를 통해서도 충분히 알 수 있겠다.   
그래서 더 경계하는 마음이 생기면서도   
다른 한편으로는 내 마음에 드는 추천시스템을 만드려면 어떻게 해야하는가 하는 새로운 질문을 나로 하여금 하게 만들기도 했다.   
다 쓰고 나니 좀 길어졌는데   
아 컨디션이 너무 안좋다. 쓰러질것 같다. 나중에 더 이어가야겠다.